# Features Extraction & Clustering. Part I.
This is the main report of our research which aim is to perform clustering of cryptocurrencies using given data. This report is splitted for three parts. First part represents initial preparing and top-level clustering based on data presence over dataset. Second part will contain both of proposed ways of features extraction and also clustering. Third part (bonus track) will include clustering with splitting timeline by states of BTC.

The problem of time series clustering can be cosidered as finding a function

$$f(X_T) = y \in [1...K]$$

$$\text{for }X_T=(x_1, ..., x_T)$$

$$\text{with }x_T \in\mathbb{R^d}$$

where T is timeline length and K is particular cluster. This should be done with representation of time serie as set of selected **features** $v_i$ of fixed size $D$ independent of $T$:

$$\phi(X_T) = v_1, ... , v_D \in \mathbb{R}$$

After that it's possible to apply standard clustering algorithms on this feature set. Main question is what features can we take into account? We have multiple time series describing each coin and also we've constructed couple of derivative parameters. 

The simple way of going from simple to complex is proposed:

1. Use pretty common, standard features for each serie (parameter): Means, Medians, Standard deviations, Skewness, Kurtosis.
2. Use [tsfresh](https://tsfresh.readthedocs.io/en/latest/index.html) library to automate process of features extraction
3. Apply both approaches to series fragmented by state of BTC.

## Preparing
One of key conclusions obtained from Data Completeness research was: *"cutting the data before Q1 of 2014 seems like a good point"*. This will be the first step. Extractor class was modified to get start date as input parameter, so tilmeline of coins which appeared earlier than passed date will be cut by this date.

Secondly, we have to take in account presence of data among coins and parameters. The main reason for this is impossibility of determine measure of similarity between coins that significantly differs in terms of data presence. Let's have a look to applicability matrix:

In [1]:
# Import of necessary libs and our classes
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from cj_loader import Storer, Extractor, extract_features
storer = Storer()

In [2]:
precense = storer.applicability()
with pd.option_context('display.max_rows', None, 'display.max_columns', 30):
    display(precense)

,active_address,close,high,low,mcap,mining_difficulty,mining_fees,nva,nvt,nvv,open,txcount,txvolume,volatility,volume,rate_btc,mcap_ratio,trad_to_trans_vol,trans_per_address,tx_per_address,mdiff_to_volatility,volatility_to_mdiff
reddcoin,False,True,True,True,True,False,False,False,False,True,True,False,False,True,True,True,True,False,False,False,False,False
nxt,False,True,True,True,True,False,False,False,False,False,True,False,False,True,True,True,True,False,False,False,False,False
bitsend,False,True,True,True,True,False,False,False,False,True,True,False,False,True,True,True,True,False,False,False,False,False
digixdao,False,True,True,True,True,False,False,False,False,True,True,False,False,True,True,True,True,False,False,False,False,False
nano,False,True,True,True,True,False,False,False,False,True,True,False,False,True,True,True,True,False,False,False,False,False
vechain,True,True,True,True,True,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False
nuls,False,True,True,True,True,False,False,False,False,True,True,False,False,True,True,True,True,False,False,False,False,False
aeternity,True,True,True,True,True,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False
blocknet,False,True,True,True,True,False,False,False,False,True,True,False,False,True,True,True,True,False,False,False,False,False
cryptonex,False,True,True,True,True,False,False,False,False,True,True,False,False,True,True,True,True,False,False,False,False,False


Definitely, precense of data itself should be taken into account as one of the first (nearest to "root" in terms of hierarchical clustering) steps of clustering. The key question is how to transform this boolean matrix to distance matrix. All parameters are different and should have different contribution to distance. In other words we can't just replace True values with 1 and False values with 0. Our goal is to replace values with *weight* in accordance to level of parameter's significance. 

It seems intuitively that the less often parameter is presented among all coins — the less affect to distance measure it should have. As good starting point we can take ratio of total parameter precense in given data and use it as weight.

In [3]:
# Calculation of data completeness matrix
data_compl = storer.data_completeness()

In [4]:
weights = data_compl.mean().sort_values()

In [5]:
from scipy.spatial.distance import squareform, pdist

# transform boolean matrix to numeric
weighted = precense.copy()
for index, row in precense.iterrows():
    weighted.loc[index][:] = row * weights

# calculate distances between pairs of coins
distances = pd.DataFrame(squareform(pdist(weighted)), index=weighted.index, columns=weighted.index)

## What clustering method to choose?
**Little disclaimer**. Since our strategy now is to build the whole pipeline that will solve the problem of cryptocurrencies clustering, it's supposed to place emphasis on fast developing that will allow easy scaling and changing some of it's parts. From this point of view DBSCAN seems the most applicable algorithm as one of the most universal. 

The DBSCAN algorithm views clusters as areas of high density separated by areas of low density. Due to this rather generic view, clusters found by DBSCAN can be any shape, as opposed to k-means which assumes that clusters are convex shaped. This is exactly the case of our quite rarified data.

The central component to the DBSCAN is the concept of core samples, which are samples that are in areas of high density. A cluster is therefore a set of core samples, each close to each other (measured by some distance measure) and a set of non-core samples that are close to a core sample (but are not themselves core samples). There are two parameters to the algorithm, min_samples and eps, which define formally what we mean when we say dense. Higher min_samples or lower eps indicate higher density necessary to form a cluster.

Additional advantage of DBSCAN is calculation of estimated number of clusters.

In [6]:
from sklearn.cluster import DBSCAN

# clustering with DBSCAN algorithm
clustering = DBSCAN(eps=0.3, min_samples=3).fit(distances)

labels = clustering.labels_
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
print('Estimated number of clusters: %d' % n_clusters_)

Estimated number of clusters: 4


Let's have a short look on resulting clusters.

In [7]:
weighted['label'] = labels
clusters = {}
for label in np.unique(labels):
    print('label %d' %label)
    cl = weighted[weighted['label'] == label]
    clusters[label] = cl
    display(cl)
    print('-'*40)
    
    

label -1


,active_address,close,high,low,mcap,mining_difficulty,mining_fees,nva,nvt,nvv,...,volatility,volume,rate_btc,mcap_ratio,trad_to_trans_vol,trans_per_address,tx_per_address,mdiff_to_volatility,volatility_to_mdiff,label
nxt,0,0.538243,0.538243,0.538243,0.896864,0.896864,0,0,0,0,...,0.538243,0,0,0,0,0,0.498049,0,0.468794,-1
smartcash,0,0.538243,0.538243,0.538243,0.896864,0.896864,0,0,0,0,...,0.538243,0,0,0,0,0,0.498049,0,0.468794,-1
stellar,0.965705,0.538243,0.538243,0.538243,0.896864,0.896864,0.486025,0.978447,0.978447,0.409819,...,0.538243,0,0,0.965113,0.965705,0,0.498049,0.486025,0.468794,-1
monero,0.965705,0.538243,0.538243,0.538243,0.896864,0.896864,0.486025,0.978447,0.978447,0.409819,...,0.538243,0,0,0.965113,0.965705,0,0.498049,0.486025,0.468794,-1
tenx,0.965705,0.538243,0.538243,0.538243,0.896864,0.896864,0,0,0,0,...,0.538243,0.436882,0.505745,0.965113,0.965705,0.505929,0.498049,0,0.468794,-1


----------------------------------------
label 0


,active_address,close,high,low,mcap,mining_difficulty,mining_fees,nva,nvt,nvv,...,volatility,volume,rate_btc,mcap_ratio,trad_to_trans_vol,trans_per_address,tx_per_address,mdiff_to_volatility,volatility_to_mdiff,label
reddcoin,0,0.538243,0.538243,0.538243,0.896864,0.896864,0,0,0,0,...,0.538243,0,0,0,0,0,0.498049,0,0.468794,0
bitsend,0,0.538243,0.538243,0.538243,0.896864,0.896864,0,0,0,0,...,0.538243,0,0,0,0,0,0.498049,0,0.468794,0
digixdao,0,0.538243,0.538243,0.538243,0.896864,0.896864,0,0,0,0,...,0.538243,0,0,0,0,0,0.498049,0,0.468794,0
nano,0,0.538243,0.538243,0.538243,0.896864,0.896864,0,0,0,0,...,0.538243,0,0,0,0,0,0.498049,0,0.468794,0
nuls,0,0.538243,0.538243,0.538243,0.896864,0.896864,0,0,0,0,...,0.538243,0,0,0,0,0,0.498049,0,0.468794,0
blocknet,0,0.538243,0.538243,0.538243,0.896864,0.896864,0,0,0,0,...,0.538243,0,0,0,0,0,0.498049,0,0.468794,0
cryptonex,0,0.538243,0.538243,0.538243,0.896864,0.896864,0,0,0,0,...,0.538243,0,0,0,0,0,0.498049,0,0.468794,0
huobi-token,0,0.538243,0.538243,0.538243,0.896864,0.896864,0,0,0,0,...,0.538243,0,0,0,0,0,0.498049,0,0.468794,0
exclusivecoin,0,0.538243,0.538243,0.538243,0.896864,0.896864,0,0,0,0,...,0.538243,0,0,0,0,0,0.498049,0,0.468794,0
theta-token,0,0.538243,0.538243,0.538243,0.896864,0.896864,0,0,0,0,...,0.538243,0,0,0,0,0,0.498049,0,0.468794,0


----------------------------------------
label 1


...

----------------------------------------
label 2


,active_address,close,high,low,mcap,mining_difficulty,mining_fees,nva,nvt,nvv,...,volatility,volume,rate_btc,mcap_ratio,trad_to_trans_vol,trans_per_address,tx_per_address,mdiff_to_volatility,volatility_to_mdiff,label
factom,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
elastos,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
gifto,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


----------------------------------------
label 3


,active_address,close,high,low,mcap,mining_difficulty,mining_fees,nva,nvt,nvv,...,volatility,volume,rate_btc,mcap_ratio,trad_to_trans_vol,trans_per_address,tx_per_address,mdiff_to_volatility,volatility_to_mdiff,label
bitcoin-gold,0.965705,0.538243,0.538243,0.538243,0.896864,0.896864,0.486025,0.978447,0.978447,0.409819,...,0.538243,0.436882,0.505745,0.965113,0.965705,0.505929,0.498049,0.486025,0.468794,3
lisk,0.965705,0.538243,0.538243,0.538243,0.896864,0.896864,0.486025,0.978447,0.978447,0.409819,...,0.538243,0.436882,0.505745,0.965113,0.965705,0.505929,0.498049,0.486025,0.468794,3
pivx,0.965705,0.538243,0.538243,0.538243,0.896864,0.896864,0.486025,0.978447,0.978447,0.409819,...,0.538243,0.436882,0.505745,0.965113,0.965705,0.505929,0.498049,0.486025,0.468794,3
bitcoin,0.965705,0.538243,0.538243,0.538243,0.896864,0.896864,0.486025,0.978447,0.978447,0.409819,...,0.538243,0.436882,0.505745,0.965113,0.965705,0.505929,0.498049,0.486025,0.468794,3
zcash,0.965705,0.538243,0.538243,0.538243,0.896864,0.896864,0.486025,0.978447,0.978447,0.409819,...,0.538243,0.436882,0.505745,0.965113,0.965705,0.505929,0.498049,0.486025,0.468794,3
neo,0.965705,0.538243,0.538243,0.538243,0.896864,0.896864,0.486025,0.978447,0.978447,0.409819,...,0.538243,0.436882,0.505745,0.965113,0.965705,0.505929,0.498049,0.486025,0.468794,3
bitcoin-cash,0.965705,0.538243,0.538243,0.538243,0.896864,0.896864,0.486025,0.978447,0.978447,0.409819,...,0.538243,0.436882,0.505745,0.965113,0.965705,0.505929,0.498049,0.486025,0.468794,3
decred,0.965705,0.538243,0.538243,0.538243,0.896864,0.896864,0.486025,0.978447,0.978447,0.409819,...,0.538243,0.436882,0.505745,0.965113,0.965705,0.505929,0.498049,0.486025,0.468794,3
gas,0.965705,0.538243,0.538243,0.538243,0.896864,0.896864,0.486025,0.978447,0.978447,0.409819,...,0.538243,0.436882,0.505745,0.965113,0.965705,0.505929,0.498049,0.486025,0.468794,3
nem,0.965705,0.538243,0.538243,0.538243,0.896864,0.896864,0.486025,0.978447,0.978447,0.409819,...,0.538243,0.436882,0.505745,0.965113,0.965705,0.505929,0.498049,0.486025,0.468794,3


----------------------------------------


As it was revealed on previous stages, there are three coins without any information (label 2 above). We shall exclude them from research. Other clusters seems similar in terms of data precense. Cluster with -1 label considered as kind of a *side* class, consisted of elements that can't be included to any of others clusters. But we will study it also.

## Features Exctractor introducing 

Now we will use a new function **extract_features** that (together with new **Extractor** class method **__features**) calculates values of basic features for each parameter. See cj_loader.py for details.

In [8]:
cl_coin_features = {}
for label, cluster in clusters.items():
    cl_coin_features[label] = extract_features(storer, '2014-04-01', coins_set=cluster.index)
    

C:\Python36\lib\site-packages\numpy\lib\function_base.py:1946: RuntimeWarning: invalid value encountered in subtract
  a = op(a[slice1], a[slice2])
C:\Python36\lib\site-packages\numpy\core\_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)
C:\Python36\lib\site-packages\scipy\interpolate\_cubic.py:317: RuntimeWarning: invalid value encountered in greater
  ind = np.nonzero(f12 > 1e-9 * np.max(f12))
C:\Python36\lib\site-packages\scipy\stats\stats.py:943: RuntimeWarning: invalid value encountered in subtract
  a_zero_mean = a - np.expand_dims(np.mean(a, axis), axis)
C:\Python36\lib\site-packages\scipy\interpolate\_cubic.py:310: RuntimeWarning: invalid value encountered in add
  t = .5 * (m[3:] + m[:-3])
C:\Python36\lib\site-packages\scipy\interpolate\_cubic.py:323: RuntimeWarning: invalid value encountered in subtract
  c = (3. * m[2:-2] - 2. * t[:-1] - t[1:]) / dx
C:\Python36\lib\site-packages\scipy\interpolate\_cubic.py

Below is result of extracting features for cluster with label 1. 

In [9]:
cl_coin_features[1]

,active_address_mean,active_address_median,active_address_stddev,active_address_skewns,active_address_kurtos,close_mean,close_median,close_stddev,close_skewns,close_kurtos,...,mdiff_to_volatility_mean,mdiff_to_volatility_median,mdiff_to_volatility_stddev,mdiff_to_volatility_skewns,mdiff_to_volatility_kurtos,volatility_to_mdiff_mean,volatility_to_mdiff_median,volatility_to_mdiff_stddev,volatility_to_mdiff_skewns,volatility_to_mdiff_kurtos
vechain,949.395,699.5,1200.31,9.38013,127.999,2.59025,2.61,2.07374,0.348733,-0.83717,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aeternity,381.744,303,445.204,10.06,137.74,1.59513,1.49,1.12034,0.755745,-0.0634447,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tron,7301.37,3237,20008.4,7.26618,64.9036,0.0398372,0.038716,0.033618,1.58926,5.62137,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
waltonchain,315.146,254,263.001,3.73752,22.2632,11.3641,9.865,7.50995,1.30666,1.93931,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ethos,323.565,278,208.521,2.36834,9.20659,2.25641,1.7,1.83529,1.70795,3.73636,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
loom-network,344.623,178.5,528.231,3.53911,12.979,0.261564,0.213449,0.144863,0.990261,0.248495,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
funfair,568.948,297,630.972,2.18021,4.64926,0.0394598,0.029683,0.0290959,2.53357,7.66513,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
omisego,3904.14,2467.5,8247.7,6.69132,49.8095,10.8949,9.82,4.86972,0.360876,0.131471,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aelf,410.697,257,1106.88,13.4756,190.262,1.09167,1.05,0.451269,0.740229,-0.127404,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
augur,456.766,377,323.064,3.1916,18.7215,20.2614,15.85,19.9044,1.71552,3.2955,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Conclusions
- Top-level clusters were obtained relying on data precense across all given coins
- Extractor of basic features was applied to each "big" cluster
- Everything is now prepared for clustering inside top-level clusters

Of course this is not the only possible approach to perform clustering of cryptocurrencies. Particulary this (quite rough) method of basic features extraction and using them as **coin profiles** can be scaled: for example features can be extracted for different periods of time, forming more wide set of features for each coin. Alternatively, measure of similarity can be found for different periods and compiled in unified metrics across all periods. 

Next step is to perform clustering relying on extracted features and additionally use tsfresh library as alternative approach.